In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import warnings
from sklearn import linear_model
import matplotlib.pyplot as plt

In [2]:
pf = pd.read_excel(r"E:\比赛相关\污染源\白酒行业参数一览表.xls", sheet_name='总表',encodig='utf8')

In [3]:
pf.head()

,单位详细名称,行业名称1,行业代码1,工业总产值(当年价格)(千元),用水量（万吨）,排水量（万吨）,是否进入污水处理厂,污染类别,原料名称,产品名称,...,污染治理设施实际运行参数二名称,污染治理设施实际运行参数二数值,污染治理设施实际运行参数三名称,污染治理设施实际运行参数三数值,污染物排放量,污染物排放量计量单位,K值计算结果,K值核算填报结果,是否有监测数据,监测法核算排放量（吨）
0,永川区清泉酒厂,白酒制造,1512,20.0,0.007,0.0063,三教工业园区污水处理厂,废水,高粱、稻米等,清香型白酒,...,污水处理设施总额定功率,4千瓦,污水处理设施耗电量,1300千瓦时/年,0.000297,吨,1.0,1.0,/,NaN
1,永川区清泉酒厂,白酒制造,1512,20.0,0.007,0.0063,三教工业园区污水处理厂,废水,高粱、稻米等,清香型白酒,...,污水处理设施总额定功率,4千瓦,污水处理设施耗电量,1300千瓦时/年,0.000015,吨,1.0,1.0,/,NaN
2,永川区清泉酒厂,白酒制造,1512,20.0,0.007,0.0063,三教工业园区污水处理厂,废水,高粱、稻米等,清香型白酒,...,污水处理设施总额定功率,4千瓦,污水处理设施耗电量,1300千瓦时/年,0.000004,吨,1.0,1.0,/,NaN
3,永川区清泉酒厂,白酒制造,1512,20.0,0.007,0.0063,三教工业园区污水处理厂,废水,高粱、稻米等,清香型白酒,...,污水处理设施总额定功率,4千瓦,污水处理设施耗电量,1300千瓦时/年,0.000001,吨,1.0,1.0,/,NaN
4,永川区清泉酒厂,白酒制造,1512,20.0,0.007,0.0063,三教工业园区污水处理厂,废气,柴油,蒸汽/热水/其他,...,NaN,NaN,NaN,NaN,0.004330,吨,NaN,1.0,/,NaN


In [4]:
pf.shape

(14384, 36)

# 特征工程

In [5]:
np.where(pf.isnull().sum()/pf.shape[0]<0.5 )[0]

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 30, 31, 33, 34], dtype=int64)

In [6]:
object_col = ['污染类别','原料名称','产品名称', '工艺名称', '生产规模等级','污染物名称','污染物处理工艺名称']  # 需要做labelencoder的列
data = pf[['污染类别','原料名称','产品名称', '工艺名称', '生产规模等级','污染物名称','污染物处理工艺名称']]
for i in object_col:
    lbl = LabelEncoder()
    data[i] = lbl.fit_transform(data[i].astype(str))
    data[i] = data[i]
data.head()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,污染类别,原料名称,产品名称,工艺名称,生产规模等级,污染物名称,污染物处理工艺名称
0,1,11,2,6,1,1,11
1,1,11,2,6,1,4,11
2,1,11,2,6,1,6,11
3,1,11,2,6,1,5,11
4,0,6,10,7,4,7,0


In [7]:
pf['污染物去除效率']= pf['污染物去除效率'].apply(lambda x: float(x.split('%')[0])*0.01)

In [8]:
train  = pf[['工业总产值(当年价格)(千元)']]
train['用水量（万吨）'] = pf['用水量（万吨）'].apply(lambda x: 0 if x=='/' else x)
train['排水量（万吨）'] = pf['排水量（万吨）'].apply(lambda x: 0 if x=='/' else x)
train['是否进入污水处理厂']= pf['是否进入污水处理厂'].map(lambda x: int(x!='/'))
train['原料/燃料用量'] = pf['原料/燃料用量'].apply(lambda x: 0 if np.isnan(x) else x)
train['污染物产生量'] = pf['污染物产生量']
train['污染物去除效率'] = pf['污染物去除效率']
columns = ['工业总产值(当年价格)(千元)', '用水量（万吨）','排水量（万吨）', '是否进入污水处理厂', '原料/燃料用量','污染物产生量','污染物去除效率']
train.head()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = val

,工业总产值(当年价格)(千元),用水量（万吨）,排水量（万吨）,是否进入污水处理厂,原料/燃料用量,污染物产生量,污染物去除效率
0,20.0,0.007,0.0063,1,0.000,0.014232,0.9791
1,20.0,0.007,0.0063,1,0.000,0.000144,0.8957
2,20.0,0.007,0.0063,1,0.000,0.000040,0.8988
3,20.0,0.007,0.0063,1,0.000,0.000028,0.9737
4,20.0,0.007,0.0063,1,1.429,0.004330,0.0000


In [9]:
train = pd.concat([train,data],axis=1)

In [10]:
pf[['污染物排放量']].describe()

,污染物排放量
count,11911.000000
mean,0.222339
std,2.217553
min,0.000000
25%,0.000613
50%,0.010290
75%,0.089600
max,182.509182


In [11]:
train['污染物排放量'] = pf['污染物排放量']

In [12]:
train.head()

,工业总产值(当年价格)(千元),用水量（万吨）,排水量（万吨）,是否进入污水处理厂,原料/燃料用量,污染物产生量,污染物去除效率,污染类别,原料名称,产品名称,工艺名称,生产规模等级,污染物名称,污染物处理工艺名称,污染物排放量
0,20.0,0.007,0.0063,1,0.000,0.014232,0.9791,1,11,2,6,1,1,11,0.000297
1,20.0,0.007,0.0063,1,0.000,0.000144,0.8957,1,11,2,6,1,4,11,0.000015
2,20.0,0.007,0.0063,1,0.000,0.000040,0.8988,1,11,2,6,1,6,11,0.000004
3,20.0,0.007,0.0063,1,0.000,0.000028,0.9737,1,11,2,6,1,5,11,0.000001
4,20.0,0.007,0.0063,1,1.429,0.004330,0.0000,0,6,10,7,4,7,0,0.004330


In [14]:
train_end = train[~train['污染物排放量'].isnull()].reset_index(drop=True)
train_end.shape
test_end = train[train['污染物排放量'].isnull()].reset_index(drop=True)
test_end.shape

(2473, 15)

In [15]:
train_end.head()

,工业总产值(当年价格)(千元),用水量（万吨）,排水量（万吨）,是否进入污水处理厂,原料/燃料用量,污染物产生量,污染物去除效率,污染类别,原料名称,产品名称,工艺名称,生产规模等级,污染物名称,污染物处理工艺名称,污染物排放量
0,20.0,0.007,0.0063,1,0.000,0.014232,0.9791,1,11,2,6,1,1,11,0.000297
1,20.0,0.007,0.0063,1,0.000,0.000144,0.8957,1,11,2,6,1,4,11,0.000015
2,20.0,0.007,0.0063,1,0.000,0.000040,0.8988,1,11,2,6,1,6,11,0.000004
3,20.0,0.007,0.0063,1,0.000,0.000028,0.9737,1,11,2,6,1,5,11,0.000001
4,20.0,0.007,0.0063,1,1.429,0.004330,0.0000,0,6,10,7,4,7,0,0.004330


In [16]:
tr_index = ~train['污染物排放量'].isnull()
train_end = train_end[tr_index].reset_index(drop=True)
y=train_end['污染物排放量'].reset_index(drop=True)
test_end = test_end[~tr_index].reset_index(drop=True)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


# 模型

In [17]:
import lightgbm as lgb
from sklearn.model_selection import KFold,StratifiedKFold,train_test_split
from sklearn.metrics import mean_squared_error,roc_auc_score

In [18]:
X_train,X_test,y_train,y_test = train_test_split(train_end.iloc[:,:-1],train_end.iloc[:,-1],test_size=0.3)

In [19]:
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

# 参数
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'auc'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}
print('开始训练...')
# 训练
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=20,
                valid_sets=lgb_eval,
                early_stopping_rounds=5)

print('开始预测...')
# 预测
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)
# 评估
print('预估结果的rmse为:')
print(mean_squared_error(y_test, y_pred) ** 0.5)

开始训练...
[1]	valid_0's l2: 17.0829	valid_0's auc: 0.777923
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's l2: 16.9777	valid_0's auc: 0.783328
[3]	valid_0's l2: 16.8793	valid_0's auc: 0.783328
[4]	valid_0's l2: 16.7873	valid_0's auc: 0.783328
[5]	valid_0's l2: 16.7011	valid_0's auc: 0.783328
[6]	valid_0's l2: 16.6611	valid_0's auc: 0.783328
[7]	valid_0's l2: 16.6242	valid_0's auc: 0.783328
Early stopping, best iteration is:
[2]	valid_0's l2: 16.9777	valid_0's auc: 0.783328
开始预测...
预估结果的rmse为:
4.12040084254308


In [20]:
pd.DataFrame({"预测值":y_pred,"实际值":y_test})

,预测值,实际值
9004,0.177693,0.000476
3760,0.302749,2.435300
8135,0.178879,0.011638
9270,0.177693,0.000781
8215,0.177693,0.001479
...,...,...
772,0.181070,0.039690
2851,0.203154,0.259200
6035,0.177693,0.004677
9533,0.181070,0.026460
